In [2]:
import pandas as pd

In [ ]:
# format change taxonomic data (from .group to .csv) 
# for now only mpa4_class.profile is considered
taxonomy_df = pd.read_csv("data/mpa4_class.profile", sep="\t")
print(taxonomy_df.head())

# format change metadata (from .group to .csv)
metadata_df = pd.read_csv("data/sample.group", sep="\t", header=None)
print(metadata_df)
print(metadata_df.info())

                     name    850945    907005    907544    907995    910252  \
0           c__Clostridia  51.07453  19.25384  34.13202  40.45823  63.65824   
1          c__Bacteroidia  42.97830  67.00596  26.10095  49.82172  26.47992   
2        c__Negativicutes   2.08592   2.70182   0.41849   2.10271   3.52240   
3  c__Gammaproteobacteria   1.28249   9.25834  36.41831   0.00000   0.42619   
4   c__Betaproteobacteria   0.93501   0.95815   0.00000   3.79642   0.46030   

     917369    920071     A0002     A0005  ...  self_CRC23  self_CRC24  \
0  43.67910  48.12092  43.68023   6.24566  ...    53.93475    34.40354   
1  44.99695  30.36404  50.71171  88.15693  ...    26.12067     7.49449   
2   2.13157   8.24484   0.47678   0.81488  ...     6.59048     1.46674   
3   5.68301   0.00579   0.00000   0.20769  ...     1.93735     0.20606   
4   0.93574   1.52207   0.34718   1.96452  ...     0.04980     0.08714   

   self_CRC26  self_CRC28  self_CRC30  self_CRC31  self_CRC34  self_CRC35  \
0  

In [4]:
# rename and match metadata and taxonomic data for merging

taxonomy_df.rename(columns={taxonomy_df.columns[0]: "Taxa"}, inplace=True)
print(taxonomy_df.head())

metadata_df.columns = [
    "SampleID", "Group", "Group_1", "Group_2", "LevelB", "LevelA",
    "GroupDetails", "Project", "Project_1", "Project_2", "Unused_1",
    "Unused_2", "Unused_3", "Age", "Gender", "BMI", "Location",
    "LocationDetails", "SequenceType", "DNAExtractType"
]
print(metadata_df.head())

# verify match
print(set(taxonomy_df.columns[1:]).difference(metadata_df["SampleID"]))

                     Taxa    850945    907005    907544    907995    910252  \
0           c__Clostridia  51.07453  19.25384  34.13202  40.45823  63.65824   
1          c__Bacteroidia  42.97830  67.00596  26.10095  49.82172  26.47992   
2        c__Negativicutes   2.08592   2.70182   0.41849   2.10271   3.52240   
3  c__Gammaproteobacteria   1.28249   9.25834  36.41831   0.00000   0.42619   
4   c__Betaproteobacteria   0.93501   0.95815   0.00000   3.79642   0.46030   

     917369    920071     A0002     A0005  ...  self_CRC23  self_CRC24  \
0  43.67910  48.12092  43.68023   6.24566  ...    53.93475    34.40354   
1  44.99695  30.36404  50.71171  88.15693  ...    26.12067     7.49449   
2   2.13157   8.24484   0.47678   0.81488  ...     6.59048     1.46674   
3   5.68301   0.00579   0.00000   0.20769  ...     1.93735     0.20606   
4   0.93574   1.52207   0.34718   1.96452  ...     0.04980     0.08714   

   self_CRC26  self_CRC28  self_CRC30  self_CRC31  self_CRC34  self_CRC35  \
0  

ValueError: Length mismatch: Expected axis has 21 elements, new values have 20 elements

In [8]:
# installing xgboost
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 59.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 58.1 MB/s eta 0:00:0000:0100:01


In [27]:
# attempt to set up XGBoost without linking taxonomic data and metadata
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load the taxonomic dataset (mpa4_class.profile)
data = pd.read_csv("data/mpa4_class.profile", sep="\t")

data.index = data.index.astype(str)

# Transpose the dataset to have samples as rows and features as columns
data_transposed = data.set_index('name').transpose()
print(data.head())

                     name    850945    907005    907544    907995    910252  \
0           c__Clostridia  51.07453  19.25384  34.13202  40.45823  63.65824   
1          c__Bacteroidia  42.97830  67.00596  26.10095  49.82172  26.47992   
2        c__Negativicutes   2.08592   2.70182   0.41849   2.10271   3.52240   
3  c__Gammaproteobacteria   1.28249   9.25834  36.41831   0.00000   0.42619   
4   c__Betaproteobacteria   0.93501   0.95815   0.00000   3.79642   0.46030   

     917369    920071     A0002     A0005  ...  self_CRC23  self_CRC24  \
0  43.67910  48.12092  43.68023   6.24566  ...    53.93475    34.40354   
1  44.99695  30.36404  50.71171  88.15693  ...    26.12067     7.49449   
2   2.13157   8.24484   0.47678   0.81488  ...     6.59048     1.46674   
3   5.68301   0.00579   0.00000   0.20769  ...     1.93735     0.20606   
4   0.93574   1.52207   0.34718   1.96452  ...     0.04980     0.08714   

   self_CRC26  self_CRC28  self_CRC30  self_CRC31  self_CRC34  self_CRC35  \
0  

In [ ]:

# Extract labels (y) - ensure you replace 'label_column' with your actual label column
labels = data.index.str.contains('self_CRC') 
y = labels.astype(int)  # Convert boolean to integer (1 for CRC, 0 for others)

# Drop the index to form the feature matrix (X)
X = data.reset_index(drop=True)
display(X)

AttributeError: Can only use .str accessor with string values!

In [35]:
X = pd.get_dummies(X, drop_first=True)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

# Convert to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set XGBoost parameters
params = {
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'logloss',       # Evaluation metric
    'max_depth': 6,                 # Maximum depth of trees
    'eta': 0.1,                     # Learning rate
    'verbosity': 1,                 # Verbosity level
    'seed': 42                      # Random seed for reproducibility
}

# Train the XGBoost model
num_round = 100
bst = xgb.train(params, dtrain, num_round)

# Predict on the test set
y_pred_prob = bst.predict(dtest)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

Train size: (148, 8626), Test size: (37, 8626)


In [30]:

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))


Accuracy: 1.00
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1721
           1       0.00      0.00      0.00         5

    accuracy                           1.00      1726
   macro avg       0.50      0.50      0.50      1726
weighted avg       0.99      1.00      1.00      1726



/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [31]:
# what do the results tell us?
# the majority class is almost exclusively predicted, while the minority class is being ignored completely
# this might be due to a mismatch of samples/entries

In [32]:
# potential fix:
# use only species level for profiling 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Load the dataset
# Replace 'data/mpa4_class.profile' with your actual file path
data = pd.read_csv('data/mpa4_class.profile', sep="\t")

# 1. Drop the 'name' column and create features (X)
X = data.drop(columns=['name'])

# 2. Create binary target labels (y)
# Mark rows as '1' if 'name' contains 'self_CRC', otherwise '0'
y = data['name'].str.contains('self_CRC').astype(int)

# 3. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Train size: {X_train.shape}, Test size: {X_test.shape}")

# 4. Convert to DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# 5. Set XGBoost parameters
params = {
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'logloss',        # Log-loss as the evaluation metric
    'max_depth': 6,                  # Tree depth
    'eta': 0.1,                      # Learning rate
    'verbosity': 1,                  # Verbose output
    'seed': 42                       # Reproducibility
}

# 6. Train the XGBoost model
num_round = 100  # Number of boosting rounds
bst = xgb.train(params, dtrain, num_round)

# 7. Make predictions on the test set
y_pred_prob = bst.predict(dtest)  # Predict probabilities
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert to binary predictions

# 8. Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))


Train size: (148, 8626), Test size: (37, 8626)
Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        37

    accuracy                           1.00        37
   macro avg       1.00      1.00      1.00        37
weighted avg       1.00      1.00      1.00        37

